<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-spring-2018) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [7]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [8]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [9]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [10]:
train.shape

(100000, 9)

In [11]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [12]:
test.shape

(100000, 8)

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [7]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

In [8]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795691465352607

In [9]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

In [6]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [13]:
from catboost import CatBoostClassifier

In [8]:
CatBoostClassifier?

In [24]:
ctb = CatBoostClassifier(depth=7, iterations=1000, learning_rate = 0.07, loss_function='Logloss')

In [16]:
train['marshrut'] = train['Origin'] + train['Dest']

In [17]:
test['marshrut'] = test['Origin'] + test['Dest']

In [18]:
train_df, y = train.drop('dep_delayed_15min', axis=1), train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [20]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [24]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

0:	learn: 0.6764806	total: 129ms	remaining: 2m 8s
1:	learn: 0.6608737	total: 193ms	remaining: 1m 36s
2:	learn: 0.6464844	total: 242ms	remaining: 1m 20s
3:	learn: 0.6332753	total: 288ms	remaining: 1m 11s
4:	learn: 0.6205674	total: 344ms	remaining: 1m 8s
5:	learn: 0.6092623	total: 421ms	remaining: 1m 9s
6:	learn: 0.5982865	total: 470ms	remaining: 1m 6s
7:	learn: 0.5884337	total: 518ms	remaining: 1m 4s
8:	learn: 0.5791218	total: 571ms	remaining: 1m 2s
9:	learn: 0.5705957	total: 641ms	remaining: 1m 3s
10:	learn: 0.5625458	total: 697ms	remaining: 1m 2s
11:	learn: 0.5551428	total: 746ms	remaining: 1m 1s
12:	learn: 0.5480880	total: 795ms	remaining: 1m
13:	learn: 0.5416483	total: 857ms	remaining: 1m
14:	learn: 0.5356460	total: 904ms	remaining: 59.4s
15:	learn: 0.5300235	total: 950ms	remaining: 58.4s
16:	learn: 0.5248868	total: 991ms	remaining: 57.3s
17:	learn: 0.5202273	total: 1.05s	remaining: 57.5s
18:	learn: 0.5157907	total: 1.1s	remaining: 56.9s
19:	learn: 0.5113949	total: 1.15s	remaining: 

164:	learn: 0.4298574	total: 9.68s	remaining: 49s
165:	learn: 0.4297713	total: 9.74s	remaining: 48.9s
166:	learn: 0.4297036	total: 9.78s	remaining: 48.8s
167:	learn: 0.4296201	total: 9.83s	remaining: 48.7s
168:	learn: 0.4295577	total: 9.88s	remaining: 48.6s
169:	learn: 0.4294645	total: 9.94s	remaining: 48.5s
170:	learn: 0.4293432	total: 9.99s	remaining: 48.4s
171:	learn: 0.4292754	total: 10s	remaining: 48.3s
172:	learn: 0.4292069	total: 10.1s	remaining: 48.2s
173:	learn: 0.4291488	total: 10.2s	remaining: 48.2s
174:	learn: 0.4290954	total: 10.2s	remaining: 48.3s
175:	learn: 0.4290413	total: 10.3s	remaining: 48.4s
176:	learn: 0.4289816	total: 10.4s	remaining: 48.5s
177:	learn: 0.4289262	total: 10.5s	remaining: 48.5s
178:	learn: 0.4288557	total: 10.5s	remaining: 48.4s
179:	learn: 0.4287897	total: 10.6s	remaining: 48.4s
180:	learn: 0.4286578	total: 10.7s	remaining: 48.5s
181:	learn: 0.4286144	total: 10.8s	remaining: 48.5s
182:	learn: 0.4284661	total: 10.9s	remaining: 48.5s
183:	learn: 0.42

324:	learn: 0.4211576	total: 19.6s	remaining: 40.7s
325:	learn: 0.4211086	total: 19.7s	remaining: 40.7s
326:	learn: 0.4210692	total: 19.7s	remaining: 40.6s
327:	learn: 0.4210475	total: 19.8s	remaining: 40.5s
328:	learn: 0.4209991	total: 19.8s	remaining: 40.4s
329:	learn: 0.4209675	total: 19.9s	remaining: 40.4s
330:	learn: 0.4209535	total: 19.9s	remaining: 40.3s
331:	learn: 0.4208966	total: 20s	remaining: 40.2s
332:	learn: 0.4208381	total: 20s	remaining: 40.1s
333:	learn: 0.4208155	total: 20.1s	remaining: 40s
334:	learn: 0.4207861	total: 20.1s	remaining: 39.9s
335:	learn: 0.4207459	total: 20.2s	remaining: 39.8s
336:	learn: 0.4206794	total: 20.2s	remaining: 39.7s
337:	learn: 0.4206542	total: 20.2s	remaining: 39.7s
338:	learn: 0.4205995	total: 20.3s	remaining: 39.6s
339:	learn: 0.4205683	total: 20.4s	remaining: 39.5s
340:	learn: 0.4205063	total: 20.4s	remaining: 39.4s
341:	learn: 0.4204386	total: 20.5s	remaining: 39.4s
342:	learn: 0.4203778	total: 20.5s	remaining: 39.3s
343:	learn: 0.4203

484:	learn: 0.4161089	total: 28.2s	remaining: 30s
485:	learn: 0.4160697	total: 28.3s	remaining: 29.9s
486:	learn: 0.4160351	total: 28.3s	remaining: 29.9s
487:	learn: 0.4160181	total: 28.4s	remaining: 29.8s
488:	learn: 0.4159966	total: 28.4s	remaining: 29.7s
489:	learn: 0.4159799	total: 28.5s	remaining: 29.7s
490:	learn: 0.4159503	total: 28.6s	remaining: 29.6s
491:	learn: 0.4159190	total: 28.6s	remaining: 29.5s
492:	learn: 0.4158986	total: 28.6s	remaining: 29.5s
493:	learn: 0.4158726	total: 28.7s	remaining: 29.4s
494:	learn: 0.4158379	total: 28.8s	remaining: 29.3s
495:	learn: 0.4157730	total: 28.8s	remaining: 29.3s
496:	learn: 0.4157414	total: 28.9s	remaining: 29.2s
497:	learn: 0.4157071	total: 28.9s	remaining: 29.2s
498:	learn: 0.4156721	total: 29s	remaining: 29.1s
499:	learn: 0.4156637	total: 29s	remaining: 29s
500:	learn: 0.4156494	total: 29.1s	remaining: 29s
501:	learn: 0.4156235	total: 29.2s	remaining: 28.9s
502:	learn: 0.4155945	total: 29.2s	remaining: 28.9s
503:	learn: 0.4155750	

643:	learn: 0.4119788	total: 39.2s	remaining: 21.7s
644:	learn: 0.4119674	total: 39.3s	remaining: 21.6s
645:	learn: 0.4119558	total: 39.3s	remaining: 21.5s
646:	learn: 0.4119311	total: 39.3s	remaining: 21.5s
647:	learn: 0.4119133	total: 39.4s	remaining: 21.4s
648:	learn: 0.4118923	total: 39.5s	remaining: 21.3s
649:	learn: 0.4118796	total: 39.5s	remaining: 21.3s
650:	learn: 0.4118518	total: 39.6s	remaining: 21.2s
651:	learn: 0.4118291	total: 39.6s	remaining: 21.1s
652:	learn: 0.4118049	total: 39.7s	remaining: 21.1s
653:	learn: 0.4117877	total: 39.7s	remaining: 21s
654:	learn: 0.4117546	total: 39.8s	remaining: 21s
655:	learn: 0.4117260	total: 39.8s	remaining: 20.9s
656:	learn: 0.4117076	total: 39.9s	remaining: 20.8s
657:	learn: 0.4116912	total: 40s	remaining: 20.8s
658:	learn: 0.4116592	total: 40s	remaining: 20.7s
659:	learn: 0.4116450	total: 40.1s	remaining: 20.6s
660:	learn: 0.4116181	total: 40.1s	remaining: 20.6s
661:	learn: 0.4116016	total: 40.2s	remaining: 20.5s
662:	learn: 0.411573

...

319:	learn: 0.4197821	total: 24.1s	remaining: 51.2s
320:	learn: 0.4197446	total: 24.2s	remaining: 51.2s
321:	learn: 0.4197054	total: 24.3s	remaining: 51.1s
322:	learn: 0.4196528	total: 24.4s	remaining: 51.1s
323:	learn: 0.4196175	total: 24.5s	remaining: 51.1s
324:	learn: 0.4195737	total: 24.6s	remaining: 51.1s
325:	learn: 0.4195549	total: 24.6s	remaining: 50.9s
326:	learn: 0.4194948	total: 24.7s	remaining: 50.9s
327:	learn: 0.4194557	total: 24.8s	remaining: 50.8s
328:	learn: 0.4193983	total: 24.9s	remaining: 50.7s
329:	learn: 0.4193647	total: 25s	remaining: 50.8s
330:	learn: 0.4193287	total: 25.1s	remaining: 50.8s
331:	learn: 0.4192665	total: 25.2s	remaining: 50.7s
332:	learn: 0.4192278	total: 25.3s	remaining: 50.6s
333:	learn: 0.4192064	total: 25.4s	remaining: 50.6s
334:	learn: 0.4191911	total: 25.4s	remaining: 50.5s
335:	learn: 0.4191708	total: 25.5s	remaining: 50.5s
336:	learn: 0.4191389	total: 25.6s	remaining: 50.4s
337:	learn: 0.4190811	total: 25.7s	remaining: 50.3s
338:	learn: 0.

479:	learn: 0.4149988	total: 36.2s	remaining: 39.3s
480:	learn: 0.4149796	total: 36.3s	remaining: 39.2s
481:	learn: 0.4149626	total: 36.5s	remaining: 39.2s
482:	learn: 0.4149249	total: 36.6s	remaining: 39.1s
483:	learn: 0.4148950	total: 36.7s	remaining: 39.1s
484:	learn: 0.4148683	total: 36.8s	remaining: 39.1s
485:	learn: 0.4148386	total: 36.9s	remaining: 39s
486:	learn: 0.4147821	total: 37s	remaining: 39s
487:	learn: 0.4147560	total: 37.1s	remaining: 38.9s
488:	learn: 0.4147241	total: 37.1s	remaining: 38.8s
489:	learn: 0.4147026	total: 37.2s	remaining: 38.7s
490:	learn: 0.4146743	total: 37.3s	remaining: 38.6s
491:	learn: 0.4146523	total: 37.3s	remaining: 38.6s
492:	learn: 0.4146312	total: 37.4s	remaining: 38.5s
493:	learn: 0.4146046	total: 37.5s	remaining: 38.4s
494:	learn: 0.4145719	total: 37.5s	remaining: 38.3s
495:	learn: 0.4145437	total: 37.6s	remaining: 38.2s
496:	learn: 0.4145241	total: 37.7s	remaining: 38.1s
497:	learn: 0.4144857	total: 37.8s	remaining: 38.1s
498:	learn: 0.4144

638:	learn: 0.4107165	total: 49.1s	remaining: 27.8s
639:	learn: 0.4106879	total: 49.2s	remaining: 27.7s
640:	learn: 0.4106653	total: 49.3s	remaining: 27.6s
641:	learn: 0.4106370	total: 49.4s	remaining: 27.5s
642:	learn: 0.4106076	total: 49.4s	remaining: 27.4s
643:	learn: 0.4105812	total: 49.5s	remaining: 27.4s
644:	learn: 0.4105698	total: 49.6s	remaining: 27.3s
645:	learn: 0.4105453	total: 49.6s	remaining: 27.2s
646:	learn: 0.4105297	total: 49.7s	remaining: 27.1s
647:	learn: 0.4105119	total: 49.8s	remaining: 27.1s
648:	learn: 0.4104928	total: 49.9s	remaining: 27s
649:	learn: 0.4104466	total: 50s	remaining: 26.9s
650:	learn: 0.4104407	total: 50s	remaining: 26.8s
651:	learn: 0.4104220	total: 50.1s	remaining: 26.8s
652:	learn: 0.4103932	total: 50.2s	remaining: 26.7s
653:	learn: 0.4103734	total: 50.3s	remaining: 26.6s
654:	learn: 0.4103573	total: 50.4s	remaining: 26.5s
655:	learn: 0.4103277	total: 50.4s	remaining: 26.4s
656:	learn: 0.4102942	total: 50.5s	remaining: 26.4s
657:	learn: 0.4102

798:	learn: 0.4066465	total: 1m	remaining: 15.3s
799:	learn: 0.4066214	total: 1m 1s	remaining: 15.3s
800:	learn: 0.4065888	total: 1m 1s	remaining: 15.2s
801:	learn: 0.4065784	total: 1m 1s	remaining: 15.1s
802:	learn: 0.4065620	total: 1m 1s	remaining: 15s
803:	learn: 0.4065472	total: 1m 1s	remaining: 15s
804:	learn: 0.4065271	total: 1m 1s	remaining: 14.9s
805:	learn: 0.4065077	total: 1m 1s	remaining: 14.8s
806:	learn: 0.4064816	total: 1m 1s	remaining: 14.7s
807:	learn: 0.4064604	total: 1m 1s	remaining: 14.6s
808:	learn: 0.4064241	total: 1m 1s	remaining: 14.5s
809:	learn: 0.4064065	total: 1m 1s	remaining: 14.5s
810:	learn: 0.4063892	total: 1m 1s	remaining: 14.4s
811:	learn: 0.4063604	total: 1m 1s	remaining: 14.3s
812:	learn: 0.4063312	total: 1m 1s	remaining: 14.2s
813:	learn: 0.4063212	total: 1m 1s	remaining: 14.2s
814:	learn: 0.4062774	total: 1m 2s	remaining: 14.1s
815:	learn: 0.4062657	total: 1m 2s	remaining: 14s
816:	learn: 0.4062401	total: 1m 2s	remaining: 13.9s
817:	learn: 0.4062132

957:	learn: 0.4031777	total: 1m 12s	remaining: 3.17s
958:	learn: 0.4031613	total: 1m 12s	remaining: 3.1s
959:	learn: 0.4031317	total: 1m 12s	remaining: 3.02s
960:	learn: 0.4031013	total: 1m 12s	remaining: 2.94s
961:	learn: 0.4030675	total: 1m 12s	remaining: 2.87s
962:	learn: 0.4030329	total: 1m 12s	remaining: 2.79s
963:	learn: 0.4030153	total: 1m 12s	remaining: 2.72s
964:	learn: 0.4029958	total: 1m 12s	remaining: 2.64s
965:	learn: 0.4029809	total: 1m 12s	remaining: 2.57s
966:	learn: 0.4029541	total: 1m 13s	remaining: 2.49s
967:	learn: 0.4029240	total: 1m 13s	remaining: 2.42s
968:	learn: 0.4028891	total: 1m 13s	remaining: 2.34s
969:	learn: 0.4028548	total: 1m 13s	remaining: 2.26s
970:	learn: 0.4028203	total: 1m 13s	remaining: 2.19s
971:	learn: 0.4027943	total: 1m 13s	remaining: 2.11s
972:	learn: 0.4027700	total: 1m 13s	remaining: 2.04s
973:	learn: 0.4027556	total: 1m 13s	remaining: 1.96s
974:	learn: 0.4027210	total: 1m 13s	remaining: 1.89s
975:	learn: 0.4026978	total: 1m 13s	remaining: 

In [26]:
np.mean(cv_scores)

0.7525825340373336

In [9]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

0:	learn: 0.6764602	total: 138ms	remaining: 2m 18s
1:	learn: 0.6607177	total: 209ms	remaining: 1m 44s
2:	learn: 0.6463154	total: 276ms	remaining: 1m 31s
3:	learn: 0.6326643	total: 338ms	remaining: 1m 24s
4:	learn: 0.6202335	total: 431ms	remaining: 1m 25s
5:	learn: 0.6089481	total: 498ms	remaining: 1m 22s
6:	learn: 0.5984867	total: 567ms	remaining: 1m 20s
7:	learn: 0.5885038	total: 633ms	remaining: 1m 18s
8:	learn: 0.5792428	total: 734ms	remaining: 1m 20s
9:	learn: 0.5706809	total: 811ms	remaining: 1m 20s
10:	learn: 0.5623558	total: 905ms	remaining: 1m 21s
11:	learn: 0.5548502	total: 973ms	remaining: 1m 20s
12:	learn: 0.5477520	total: 1.03s	remaining: 1m 18s
13:	learn: 0.5413434	total: 1.09s	remaining: 1m 16s
14:	learn: 0.5354962	total: 1.15s	remaining: 1m 15s
15:	learn: 0.5298009	total: 1.22s	remaining: 1m 14s
16:	learn: 0.5244013	total: 1.28s	remaining: 1m 14s
17:	learn: 0.5193351	total: 1.34s	remaining: 1m 13s
18:	learn: 0.5147928	total: 1.43s	remaining: 1m 13s
19:	learn: 0.5106658	t

160:	learn: 0.4275414	total: 13.5s	remaining: 1m 10s
161:	learn: 0.4274671	total: 13.6s	remaining: 1m 10s
162:	learn: 0.4274061	total: 13.7s	remaining: 1m 10s
163:	learn: 0.4273130	total: 13.8s	remaining: 1m 10s
164:	learn: 0.4272229	total: 13.8s	remaining: 1m 10s
165:	learn: 0.4271245	total: 13.9s	remaining: 1m 9s
166:	learn: 0.4270747	total: 14s	remaining: 1m 9s
167:	learn: 0.4269867	total: 14.2s	remaining: 1m 10s
168:	learn: 0.4269152	total: 14.3s	remaining: 1m 10s
169:	learn: 0.4268171	total: 14.4s	remaining: 1m 10s
170:	learn: 0.4267574	total: 14.5s	remaining: 1m 10s
171:	learn: 0.4266666	total: 14.6s	remaining: 1m 10s
172:	learn: 0.4266098	total: 14.7s	remaining: 1m 10s
173:	learn: 0.4264860	total: 14.8s	remaining: 1m 10s
174:	learn: 0.4264263	total: 14.8s	remaining: 1m 9s
175:	learn: 0.4263816	total: 14.9s	remaining: 1m 9s
176:	learn: 0.4262705	total: 15s	remaining: 1m 9s
177:	learn: 0.4261594	total: 15s	remaining: 1m 9s
178:	learn: 0.4260758	total: 15.1s	remaining: 1m 9s
179:	l

320:	learn: 0.4185886	total: 27.9s	remaining: 59.1s
321:	learn: 0.4185762	total: 28s	remaining: 59s
322:	learn: 0.4185177	total: 28.1s	remaining: 58.9s
323:	learn: 0.4184824	total: 28.2s	remaining: 58.9s
324:	learn: 0.4184394	total: 28.3s	remaining: 58.8s
325:	learn: 0.4184050	total: 28.4s	remaining: 58.6s
326:	learn: 0.4183641	total: 28.4s	remaining: 58.5s
327:	learn: 0.4183327	total: 28.5s	remaining: 58.3s
328:	learn: 0.4182736	total: 28.5s	remaining: 58.2s
329:	learn: 0.4182149	total: 28.6s	remaining: 58.1s
330:	learn: 0.4181754	total: 28.7s	remaining: 58.1s
331:	learn: 0.4181583	total: 28.8s	remaining: 58s
332:	learn: 0.4180825	total: 28.9s	remaining: 57.8s
333:	learn: 0.4180184	total: 28.9s	remaining: 57.7s
334:	learn: 0.4179918	total: 29s	remaining: 57.6s
335:	learn: 0.4179522	total: 29.1s	remaining: 57.4s
336:	learn: 0.4179117	total: 29.1s	remaining: 57.3s
337:	learn: 0.4178505	total: 29.2s	remaining: 57.2s
338:	learn: 0.4177961	total: 29.3s	remaining: 57.1s
339:	learn: 0.417770

481:	learn: 0.4137337	total: 43s	remaining: 46.2s
482:	learn: 0.4137121	total: 43.1s	remaining: 46.2s
483:	learn: 0.4136943	total: 43.2s	remaining: 46.1s
484:	learn: 0.4136672	total: 43.3s	remaining: 46s
485:	learn: 0.4136492	total: 43.4s	remaining: 45.9s
486:	learn: 0.4136061	total: 43.5s	remaining: 45.8s
487:	learn: 0.4135936	total: 43.5s	remaining: 45.7s
488:	learn: 0.4135562	total: 43.6s	remaining: 45.6s
489:	learn: 0.4135436	total: 43.8s	remaining: 45.6s
490:	learn: 0.4135337	total: 43.9s	remaining: 45.5s
491:	learn: 0.4135162	total: 44s	remaining: 45.4s
492:	learn: 0.4134750	total: 44.1s	remaining: 45.4s
493:	learn: 0.4134635	total: 44.2s	remaining: 45.3s
494:	learn: 0.4134294	total: 44.4s	remaining: 45.3s
495:	learn: 0.4134021	total: 44.6s	remaining: 45.3s
496:	learn: 0.4133817	total: 44.6s	remaining: 45.2s
497:	learn: 0.4133596	total: 44.7s	remaining: 45s
498:	learn: 0.4133292	total: 44.8s	remaining: 44.9s
499:	learn: 0.4133153	total: 44.8s	remaining: 44.8s
500:	learn: 0.413298

641:	learn: 0.4103718	total: 55.4s	remaining: 30.9s
642:	learn: 0.4103491	total: 55.4s	remaining: 30.8s
643:	learn: 0.4103335	total: 55.5s	remaining: 30.7s
644:	learn: 0.4103156	total: 55.6s	remaining: 30.6s
645:	learn: 0.4102863	total: 55.6s	remaining: 30.5s
646:	learn: 0.4102657	total: 55.7s	remaining: 30.4s
647:	learn: 0.4102419	total: 55.8s	remaining: 30.3s
648:	learn: 0.4102005	total: 55.9s	remaining: 30.2s
649:	learn: 0.4101828	total: 56s	remaining: 30.1s
650:	learn: 0.4101692	total: 56s	remaining: 30s
651:	learn: 0.4101520	total: 56.1s	remaining: 29.9s
652:	learn: 0.4101334	total: 56.1s	remaining: 29.8s
653:	learn: 0.4101247	total: 56.2s	remaining: 29.7s
654:	learn: 0.4100832	total: 56.3s	remaining: 29.7s
655:	learn: 0.4100645	total: 56.4s	remaining: 29.6s
656:	learn: 0.4100579	total: 56.5s	remaining: 29.5s
657:	learn: 0.4100354	total: 56.6s	remaining: 29.4s
658:	learn: 0.4100175	total: 56.7s	remaining: 29.3s
659:	learn: 0.4099944	total: 56.8s	remaining: 29.2s
660:	learn: 0.4099

800:	learn: 0.4070762	total: 1m 8s	remaining: 17.1s
801:	learn: 0.4070554	total: 1m 8s	remaining: 17s
802:	learn: 0.4070421	total: 1m 8s	remaining: 16.9s
803:	learn: 0.4070188	total: 1m 8s	remaining: 16.8s
804:	learn: 0.4070120	total: 1m 9s	remaining: 16.7s
805:	learn: 0.4069904	total: 1m 9s	remaining: 16.6s
806:	learn: 0.4069731	total: 1m 9s	remaining: 16.6s
807:	learn: 0.4069465	total: 1m 9s	remaining: 16.5s
808:	learn: 0.4069271	total: 1m 9s	remaining: 16.4s
809:	learn: 0.4068917	total: 1m 9s	remaining: 16.3s
810:	learn: 0.4068793	total: 1m 9s	remaining: 16.2s
811:	learn: 0.4068572	total: 1m 9s	remaining: 16.2s
812:	learn: 0.4068538	total: 1m 9s	remaining: 16.1s
813:	learn: 0.4068408	total: 1m 9s	remaining: 16s
814:	learn: 0.4068279	total: 1m 9s	remaining: 15.9s
815:	learn: 0.4068167	total: 1m 10s	remaining: 15.8s
816:	learn: 0.4068107	total: 1m 10s	remaining: 15.7s
817:	learn: 0.4067953	total: 1m 10s	remaining: 15.6s
818:	learn: 0.4067817	total: 1m 10s	remaining: 15.5s
819:	learn: 

956:	learn: 0.4041579	total: 1m 19s	remaining: 3.59s
957:	learn: 0.4041433	total: 1m 19s	remaining: 3.5s
958:	learn: 0.4041296	total: 1m 19s	remaining: 3.42s
959:	learn: 0.4041073	total: 1m 20s	remaining: 3.33s
960:	learn: 0.4040879	total: 1m 20s	remaining: 3.25s
961:	learn: 0.4040759	total: 1m 20s	remaining: 3.17s
962:	learn: 0.4040597	total: 1m 20s	remaining: 3.08s
963:	learn: 0.4040503	total: 1m 20s	remaining: 3s
964:	learn: 0.4040192	total: 1m 20s	remaining: 2.92s
965:	learn: 0.4040050	total: 1m 20s	remaining: 2.83s
966:	learn: 0.4039856	total: 1m 20s	remaining: 2.75s
967:	learn: 0.4039570	total: 1m 20s	remaining: 2.67s
968:	learn: 0.4039321	total: 1m 20s	remaining: 2.58s
969:	learn: 0.4039176	total: 1m 20s	remaining: 2.5s
970:	learn: 0.4039029	total: 1m 20s	remaining: 2.42s
971:	learn: 0.4038826	total: 1m 21s	remaining: 2.33s
972:	learn: 0.4038631	total: 1m 21s	remaining: 2.25s
973:	learn: 0.4038258	total: 1m 21s	remaining: 2.17s
974:	learn: 0.4037989	total: 1m 21s	remaining: 2.09

In [10]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.7643619843286743

In [11]:
catboost_test_pred = ctb.predict_proba(valid_df)[:, 1]

In [25]:
%%time
ctb.fit(train_df, y,
        cat_features=cat_features_idx);

0:	learn: 0.6540873	total: 135ms	remaining: 2m 14s
1:	learn: 0.6224357	total: 236ms	remaining: 1m 57s
2:	learn: 0.5942936	total: 369ms	remaining: 2m 2s
3:	learn: 0.5714000	total: 505ms	remaining: 2m 5s
4:	learn: 0.5520314	total: 664ms	remaining: 2m 12s
5:	learn: 0.5362845	total: 779ms	remaining: 2m 9s
6:	learn: 0.5215491	total: 897ms	remaining: 2m 7s
7:	learn: 0.5103645	total: 1.04s	remaining: 2m 8s
8:	learn: 0.4995378	total: 1.21s	remaining: 2m 12s
9:	learn: 0.4914761	total: 1.39s	remaining: 2m 17s
10:	learn: 0.4842384	total: 1.54s	remaining: 2m 18s
11:	learn: 0.4775839	total: 1.64s	remaining: 2m 15s
12:	learn: 0.4721062	total: 1.8s	remaining: 2m 17s
13:	learn: 0.4678337	total: 1.96s	remaining: 2m 17s
14:	learn: 0.4638064	total: 2.1s	remaining: 2m 17s
15:	learn: 0.4599911	total: 2.21s	remaining: 2m 16s
16:	learn: 0.4568396	total: 2.38s	remaining: 2m 17s
17:	learn: 0.4540500	total: 2.53s	remaining: 2m 18s
18:	learn: 0.4515159	total: 2.64s	remaining: 2m 16s
19:	learn: 0.4492103	total: 2

158:	learn: 0.4098321	total: 23s	remaining: 2m 1s
159:	learn: 0.4097656	total: 23.3s	remaining: 2m 2s
160:	learn: 0.4096988	total: 23.5s	remaining: 2m 2s
161:	learn: 0.4096500	total: 23.6s	remaining: 2m 1s
162:	learn: 0.4096158	total: 23.7s	remaining: 2m 1s
163:	learn: 0.4094834	total: 23.9s	remaining: 2m 1s
164:	learn: 0.4094123	total: 24s	remaining: 2m 1s
165:	learn: 0.4092831	total: 24.1s	remaining: 2m 1s
166:	learn: 0.4092173	total: 24.3s	remaining: 2m
167:	learn: 0.4091209	total: 24.4s	remaining: 2m
168:	learn: 0.4090418	total: 24.5s	remaining: 2m
169:	learn: 0.4089651	total: 24.6s	remaining: 2m
170:	learn: 0.4089080	total: 24.8s	remaining: 2m
171:	learn: 0.4088290	total: 25s	remaining: 2m
172:	learn: 0.4086742	total: 25.2s	remaining: 2m
173:	learn: 0.4085937	total: 25.3s	remaining: 2m
174:	learn: 0.4085245	total: 25.5s	remaining: 1m 59s
175:	learn: 0.4083982	total: 25.6s	remaining: 1m 59s
176:	learn: 0.4083379	total: 25.7s	remaining: 1m 59s
177:	learn: 0.4082774	total: 25.9s	rema

315:	learn: 0.3996070	total: 46.3s	remaining: 1m 40s
316:	learn: 0.3995395	total: 46.4s	remaining: 1m 39s
317:	learn: 0.3995085	total: 46.5s	remaining: 1m 39s
318:	learn: 0.3994713	total: 46.7s	remaining: 1m 39s
319:	learn: 0.3994229	total: 46.9s	remaining: 1m 39s
320:	learn: 0.3993483	total: 47.1s	remaining: 1m 39s
321:	learn: 0.3992815	total: 47.2s	remaining: 1m 39s
322:	learn: 0.3992343	total: 47.4s	remaining: 1m 39s
323:	learn: 0.3991973	total: 47.5s	remaining: 1m 39s
324:	learn: 0.3991608	total: 47.6s	remaining: 1m 38s
325:	learn: 0.3991149	total: 47.8s	remaining: 1m 38s
326:	learn: 0.3990493	total: 47.9s	remaining: 1m 38s
327:	learn: 0.3989996	total: 48s	remaining: 1m 38s
328:	learn: 0.3989060	total: 48.2s	remaining: 1m 38s
329:	learn: 0.3988840	total: 48.3s	remaining: 1m 38s
330:	learn: 0.3988292	total: 48.4s	remaining: 1m 37s
331:	learn: 0.3987950	total: 48.5s	remaining: 1m 37s
332:	learn: 0.3987010	total: 48.7s	remaining: 1m 37s
333:	learn: 0.3986175	total: 48.8s	remaining: 1m

472:	learn: 0.3908719	total: 1m 7s	remaining: 1m 14s
473:	learn: 0.3908293	total: 1m 7s	remaining: 1m 14s
474:	learn: 0.3907586	total: 1m 7s	remaining: 1m 14s
475:	learn: 0.3907238	total: 1m 7s	remaining: 1m 14s
476:	learn: 0.3906559	total: 1m 7s	remaining: 1m 14s
477:	learn: 0.3905681	total: 1m 7s	remaining: 1m 14s
478:	learn: 0.3904930	total: 1m 8s	remaining: 1m 14s
479:	learn: 0.3904427	total: 1m 8s	remaining: 1m 14s
480:	learn: 0.3904088	total: 1m 8s	remaining: 1m 13s
481:	learn: 0.3903524	total: 1m 8s	remaining: 1m 13s
482:	learn: 0.3902855	total: 1m 8s	remaining: 1m 13s
483:	learn: 0.3902359	total: 1m 8s	remaining: 1m 13s
484:	learn: 0.3902028	total: 1m 9s	remaining: 1m 13s
485:	learn: 0.3901767	total: 1m 9s	remaining: 1m 13s
486:	learn: 0.3901100	total: 1m 9s	remaining: 1m 13s
487:	learn: 0.3900508	total: 1m 9s	remaining: 1m 12s
488:	learn: 0.3899551	total: 1m 9s	remaining: 1m 12s
489:	learn: 0.3899288	total: 1m 9s	remaining: 1m 12s
490:	learn: 0.3898835	total: 1m 9s	remaining: 

627:	learn: 0.3830713	total: 1m 28s	remaining: 52.4s
628:	learn: 0.3830221	total: 1m 28s	remaining: 52.2s
629:	learn: 0.3829849	total: 1m 28s	remaining: 52.1s
630:	learn: 0.3829089	total: 1m 28s	remaining: 51.9s
631:	learn: 0.3828702	total: 1m 28s	remaining: 51.8s
632:	learn: 0.3828238	total: 1m 29s	remaining: 51.6s
633:	learn: 0.3827796	total: 1m 29s	remaining: 51.5s
634:	learn: 0.3827397	total: 1m 29s	remaining: 51.3s
635:	learn: 0.3826764	total: 1m 29s	remaining: 51.2s
636:	learn: 0.3826111	total: 1m 29s	remaining: 51s
637:	learn: 0.3825485	total: 1m 29s	remaining: 50.9s
638:	learn: 0.3824951	total: 1m 29s	remaining: 50.8s
639:	learn: 0.3824511	total: 1m 30s	remaining: 50.6s
640:	learn: 0.3824172	total: 1m 30s	remaining: 50.5s
641:	learn: 0.3823568	total: 1m 30s	remaining: 50.4s
642:	learn: 0.3823136	total: 1m 30s	remaining: 50.2s
643:	learn: 0.3822706	total: 1m 30s	remaining: 50.1s
644:	learn: 0.3822148	total: 1m 30s	remaining: 50s
645:	learn: 0.3821597	total: 1m 30s	remaining: 49.

784:	learn: 0.3750180	total: 1m 51s	remaining: 30.6s
785:	learn: 0.3749770	total: 1m 51s	remaining: 30.4s
786:	learn: 0.3749234	total: 1m 51s	remaining: 30.3s
787:	learn: 0.3748419	total: 1m 52s	remaining: 30.1s
788:	learn: 0.3747787	total: 1m 52s	remaining: 30s
789:	learn: 0.3747381	total: 1m 52s	remaining: 29.9s
790:	learn: 0.3747172	total: 1m 52s	remaining: 29.8s
791:	learn: 0.3746740	total: 1m 52s	remaining: 29.6s
792:	learn: 0.3746135	total: 1m 53s	remaining: 29.5s
793:	learn: 0.3745627	total: 1m 53s	remaining: 29.4s
794:	learn: 0.3745437	total: 1m 53s	remaining: 29.2s
795:	learn: 0.3744950	total: 1m 53s	remaining: 29.1s
796:	learn: 0.3744355	total: 1m 53s	remaining: 28.9s
797:	learn: 0.3743581	total: 1m 53s	remaining: 28.8s
798:	learn: 0.3743325	total: 1m 53s	remaining: 28.7s
799:	learn: 0.3742676	total: 1m 54s	remaining: 28.5s
800:	learn: 0.3742177	total: 1m 54s	remaining: 28.4s
801:	learn: 0.3741586	total: 1m 54s	remaining: 28.2s
802:	learn: 0.3740659	total: 1m 54s	remaining: 2

941:	learn: 0.3677893	total: 2m 16s	remaining: 8.38s
942:	learn: 0.3677295	total: 2m 16s	remaining: 8.24s
943:	learn: 0.3677127	total: 2m 16s	remaining: 8.1s
944:	learn: 0.3676580	total: 2m 16s	remaining: 7.96s
945:	learn: 0.3676117	total: 2m 16s	remaining: 7.82s
946:	learn: 0.3675608	total: 2m 17s	remaining: 7.67s
947:	learn: 0.3675392	total: 2m 17s	remaining: 7.53s
948:	learn: 0.3674878	total: 2m 17s	remaining: 7.38s
949:	learn: 0.3674397	total: 2m 17s	remaining: 7.24s
950:	learn: 0.3673983	total: 2m 17s	remaining: 7.1s
951:	learn: 0.3673492	total: 2m 17s	remaining: 6.95s
952:	learn: 0.3673202	total: 2m 18s	remaining: 6.81s
953:	learn: 0.3672830	total: 2m 18s	remaining: 6.66s
954:	learn: 0.3672644	total: 2m 18s	remaining: 6.52s
955:	learn: 0.3672307	total: 2m 18s	remaining: 6.38s
956:	learn: 0.3671716	total: 2m 18s	remaining: 6.23s
957:	learn: 0.3671317	total: 2m 18s	remaining: 6.09s
958:	learn: 0.3670570	total: 2m 19s	remaining: 5.94s
959:	learn: 0.3670213	total: 2m 19s	remaining: 5

In [26]:
catboost_test_pred = ctb.predict_proba(test)[:, 1]

In [27]:
pd.Series(catboost_test_pred, name='dep_delayed_15min').to_csv('catboost_initial.csv', index_label='id', header=True)

Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!